In [2]:
import torch
print(torch.cuda.is_available()) # 是否可以用cuda
print(torch.cuda.is_bf16_supported()) # 是否可以用bf16

True
True


dataclass用法

In [4]:
# 定义参数
from dataclasses import dataclass

@dataclass
class arg:
    x: int = 2
    y: int = 4

print(arg.x)
print(type(arg))

2
<class 'type'>


矩阵对位相乘和行列相乘的几何意义

In [2]:
import torch

A = torch.tensor([[1, 2],
                  [3, 4]])

B = torch.tensor([[5, 6],
                  [7, 8]])

print('对位相乘：\n',torch.mul(A, B))
print('对位相乘：\n',A * B)

print('行列相乘：几何意义为将向量进行旋转拉伸\n',torch.matmul(A, B))
print('行列相乘：几何意义为将向量进行旋转拉伸\n',A @ B)

对位相乘：
 tensor([[ 5, 12],
        [21, 32]])
对位相乘：
 tensor([[ 5, 12],
        [21, 32]])
行列相乘：几何意义为将向量进行旋转拉伸
 tensor([[19, 22],
        [43, 50]])
行列相乘：几何意义为将向量进行旋转拉伸
 tensor([[19, 22],
        [43, 50]])


判断矩阵形状的两种方法

In [20]:
import torch
a = torch.zeros(2,3,4)
print(a.shape)
print(a.size())

torch.Size([2, 3, 4])
torch.Size([2, 3, 4])


embedding权重

In [2]:
import torch
import torch.nn as nn

# 输入
input = torch.LongTensor([1, 2, 5, 4])

# 构建权重
vocab_size = 10000 # 词表大小
embedding_dim = 300 # 映射维度
embedding_weights = nn.Embedding(vocab_size, embedding_dim)

# 输出
embeddings = embedding_weights(input)

# embedding作用
'''
vocab_size代表有多少种不同的个体
从而为每一个个体设置了一个n个维度的向量数值，代表他们的意义或标签
从而增加一个维度
'''
print(embeddings.shape)

torch.Size([4, 300])


linear权重

In [5]:
'''
只对矩阵最后一个维度进行计算

Linear：生成线性y=wx+b权重参数的方法

行 代表有多少个线性方程
列 代表有多少个x输出
'''
# (5个输入，3个输出)
# (3行，5列)
# 举例子：(3,5) @ (5).T = (3)
linear = nn.Linear(5,3)
print('线性层的形状:',linear.weight.shape)

线性层的形状: torch.Size([3, 5])


unsqueeze增加维度

In [3]:
'''
什么是unsqueeze(x)？
-> 在x之前增加一个维度，作为新的维度。
-> 这会改变原始矩阵的形状，
原始矩阵a(2,3) 2行，3列
a.unsqueeze(0) -> (1,2,3) 1个，2行，3列
a.unsqueeze(1) -> (2,1,3) 2个，1行，3列
a.unsqueeze(2) -> (2,3,1) 2个，3行，1列

最后两个参数永远被解读为 行，列
前边的参数永远被优先解读为 分组，个数，批量 等概念
'''
a=torch.zeros(2,3)
print('原始矩阵：\n',a)
print('第一个维度左边增加：\n',a.unsqueeze(0)) # (1,2,3)
print('第二个维度左边增加：\n',a.unsqueeze(1)) # (2,1,3)
print('第三个维度左边增加：\n',a.unsqueeze(2)) # (2,3,1)

原始矩阵：
 tensor([[0., 0., 0.],
        [0., 0., 0.]])
第一个维度左边增加：
 tensor([[[0., 0., 0.],
         [0., 0., 0.]]])
第二个维度左边增加：
 tensor([[[0., 0., 0.]],

        [[0., 0., 0.]]])
第三个维度左边增加：
 tensor([[[0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.]]])


交换矩阵维度：transpose，permute两种方法

In [10]:
a = torch.zeros(2,3,4)
print(a.shape)

# permute 方法交换维度
b = a.permute(0,2,1)
print(b.shape)

# transpose 方法交换维度
c = a.transpose(0,1)
print(c.shape)

torch.Size([2, 3, 4])
torch.Size([2, 4, 3])
torch.Size([3, 2, 4])


改变矩阵形状：reshape，view两种方法

In [13]:
a = torch.zeros(2,3,4)
print(a.shape)

b = a.reshape(6,4)
print(b.shape)

c = a.view(6,4)
print(b.shape)

torch.Size([2, 3, 4])
torch.Size([6, 4])
torch.Size([6, 4])


ModuleDict保存模型参数

In [17]:
# 把模型用字典的形式保存起来
import torch.nn as nn

model = nn.ModuleDict(dict(
    a = nn.Linear(4, 2),
    b = nn.Linear(2, 5),
))
print(type(model))
print(model.a)

# 输入
input = torch.randn(1, 4)

# 顺序运算
output = input
for model,layer in model.items():
    output = layer(output)
print(output)

<class 'torch.nn.modules.container.ModuleDict'>
Linear(in_features=4, out_features=2, bias=True)
tensor([[ 0.5797,  0.3375,  0.1751,  0.5144, -0.1534]],
       grad_fn=<AddmmBackward0>)


ModuleList保存模型参数

In [18]:
import torch.nn as nn

model = nn.ModuleList([
    nn.Linear(4,2),
    nn.Linear(2,5),
])
print(type(model))

# 输入
input = torch.randn(1, 4)

# 顺序运算
output = input
for layer in model:
    output = layer(output)
print(output)

<class 'torch.nn.modules.container.ModuleList'>
tensor([[ 0.3161,  0.1486, -0.5677, -0.6741, -0.4152]],
       grad_fn=<AddmmBackward0>)


Sequential保存模型参数

In [19]:
import torch
import torch.nn as nn

model = nn.Sequential(
    nn.Linear(10, 5),  # 输入层到隐藏层
    nn.ReLU(),         # 激活函数
    nn.Linear(5, 2)    # 隐藏层到输出层
)
print(type(model))

# 输入
input = torch.randn(1, 10)

# 运算
output = model(input)
print(output)

<class 'torch.nn.modules.container.Sequential'>
tensor([[ 0.4346, -0.0176]], grad_fn=<AddmmBackward0>)


isinstance判断类别属性

In [ ]:
# 定义一个类
class a:
    pass

# 实例化的类
obj = a()

# 判断是否是某个类别
result = isinstance(obj, a)

print(result)

In [15]:
import torch
import torch.nn as nn
from torch.nn import functional as F

from arg import arg
class FeedForwardNetwork(nn.Module):
    def __init__(self,arg):
        super().__init__()

        # self.arg = arg  # 输入所有参数

        self.ffn = nn.Sequential( # 按顺序执行
            nn.Linear(arg.d_model,arg.d_model*4),
            nn.ReLU(),
            nn.Linear(arg.d_model*4,arg.d_model),

            nn.Dropout(arg.dropout)
        )
        print(arg.d_model)

    def forward(self,x):
        return self.ffn(x)

a = FeedForwardNetwork(arg)


1024
